# Welcome to PySyft

The goal of this notebook is to provide step by step explanation of the internal workings of PySyft for developers and have working examples of the API to play with.

**Note:** You should be able to run these without any issues. This notebook will be automatically run by CI and flagged if it fails. If your commit breaks this notebook, either fix the issue or add some information here for others.

In [1]:
assert True is True

In [2]:
import pytest
import json
import syft as sy
from syft.core.node.common.service.auth import AuthorizationException
from syft.util import key_emoji

/Users/madhavajay/.local/share/virtualenvs/PySyft-lHlz_cKe/lib/python3.8/site-packages/torch/distributed/distributed_c10d.py:125: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn("torch.distributed.reduce_op is deprecated, please use "


Bob decides has some data he wants to share with Alice using PySyft.
The first thing Bob needs to do is to create a Node to handle all the PySyft services he will need to safely and securely share his data with Alice.

In [4]:
somedevice = sy.Device()

This is a device, it is a Node and it has a name, id and address.

In [5]:
print(somedevice.name, somedevice.id, somedevice.address)

Flamboyant Bhaskara <UID:1cb62232-8a31-4922-977b-97d403047490> <Address - Device:<SpecificLocation:..47490>>


The ID is a class called UID which is essentially a uuid. The address is a combination of up to four different locations, identifying the path to resolve the final target of the address.

In [6]:
print(somedevice.address.vm, somedevice.address.device, somedevice.address.domain, somedevice.address.network)
print(somedevice.address.target_id)

None <SpecificLocation:1cb62232-8a31-4922-977b-97d403047490> None None
<SpecificLocation:1cb62232-8a31-4922-977b-97d403047490>


UIDs are hard to read and compare so we have a function which converts them to 2 x emoji.
Just like the "name", Emoji uniqueness is not guaranteed but is very useful during debugging.

In [7]:
print(somedevice.address.target_id.pprint)
print(somedevice.address.pprint)

📌 Flamboyant Bhaskara (SpecificLocation)@<UID:🙖🙉>
💠 [📱] Flamboyant Bhaskara (Address)


Most things that are "pretty printed" include an emoji, a name and a class, additionally with a UID and "visual UID" emoji.  
📌 somedevice.address.target_id is a "Location" which is pointing to this specific location with a name and address.  
💠 [📱] somedevice.address is the "Address" of somedevice (Think up to 4 x locations) in this case the contents of the List [] show it only has the Location of a device currently.

**note:** Sometimes Emoji's look like this: 🙓. The dynamically generated code point doesnt have an Emoji. A PR fix would be welcome. 🙂

In [8]:
print(somedevice.id, somedevice.address.target_id.id)
print(somedevice.id.emoji(), "==", somedevice.address.target_id.id.emoji())
print(somedevice.pprint)
assert somedevice.id == somedevice.address.target_id.id

<UID:1cb62232-8a31-4922-977b-97d403047490> <UID:1cb62232-8a31-4922-977b-97d403047490>
<UID:🙖🙉> == <UID:🙖🙉>
📱 Flamboyant Bhaskara (Device)@<UID:🙖🙉>


Interaction with a Node like a device is always done through a client. Clients can "send" messages and Nodes can "receive" them. Bob needs to get a client for his device. But first it might be a good idea to name is device so that its easier to follow.

In [9]:
bob_device = sy.Device(name="Bob's iPhone")
assert bob_device.name == "Bob's iPhone"

> Creating 🗃️ (MemoryStore) {}
> Creating 📱 Bob's iPhone (Device)@<UID:🚱🚃>


In [10]:
bob_device_client = bob_device.get_client()

> Creating 📡 [📱] Bob's iPhone Client (DeviceClient)@<UID:🚱🚃>


When you ask a node for a client you get a "Client" which is named after the device and has the same "UID" and "Address" (4 x Locations) as the device it was created from, and it will have a "Route" that connects it to the "Device"

In [11]:
assert bob_device_client.name == "Bob's iPhone Client"

In [12]:
print(bob_device_client.pprint, bob_device.pprint)
print(bob_device.id.emoji(), "==", bob_device_client.id.emoji())
assert bob_device.id == bob_device_client.device.id
assert bob_device.address == bob_device_client.address

📡 [📱] Bob's iPhone Client (DeviceClient)@<UID:🚱🚃> 📱 Bob's iPhone (Device)@<UID:🚱🚃>
<UID:🚱🚃> == <UID:🚱🚃>


📡 [📱] Bob's iPhone Client is a "DeviceClient" and it has the same UID and "Address" as Bob's Device

Now we have something that can send and receive lets take it for a spin.
Since everything is handled with a layer of abstraction the smallest unit of work is a "SyftMessage". Very little can be done without sending a message from a Client to a Node. There are many types of "SyftMessage" which boil down to whether or not they are Sync or Async, and whether or not they expect a response.

Lets make a ReprMessage which simply gets a message and prints it at its destination Node.
SyftMessage's all have an "address" field, without this they would never get delivered. They also generally have a msg_id which can be used to keep track of them.

In [13]:
msg = sy.ReprMessage(address=bob_device_client.address)
print(msg.pprint)
print(bob_device_client.address.pprint)
assert msg.address == bob_device_client.address

> Creating ✉️  (ReprMessage) <UID:🚰🚆>
✉️  (ReprMessage)
💠 [📱] Bob's iPhone Client (Address)


What type of Message is ReprMessage you ask?

In [14]:
print(sy.ReprMessage.mro())

[<class 'syft.core.node.common.service.repr_service.ReprMessage'>, <class 'syft.core.common.message.ImmediateSyftMessageWithoutReply'>, <class 'syft.core.common.message.ImmediateSyftMessage'>, <class 'syft.core.common.message.SyftMessageWithoutReply'>, <class 'syft.core.common.message.SyftMessage'>, <class 'syft.core.common.message.AbstractMessage'>, <class 'syft.core.common.object.ObjectWithID'>, <class 'syft.core.common.serde.serializable.Serializable'>, <class 'typing.Generic'>, <class 'object'>]


Its an "Immediate" "WithoutReply" okay so Sync and no response.

Now lets send it, remember we need a Client not a Node for sending.

In [15]:
with pytest.raises(AuthorizationException):
    bob_device_client.send_immediate_msg_without_reply(
        msg=sy.ReprMessage(address=bob_device_client.address)
    )

> Creating ✉️  (ReprMessage) <UID:🚾🚫>
> 📡 [📱] Bob's iPhone Client (DeviceClient)@<UID:🚱🚃> Signing ✉️  (ReprMessage) with 🛩
> Signing with 🛩
> Creating Signed ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) <UID:🚾🚫>
> Sending ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) 📡 [📱] Bob's iPhone Client (DeviceClient)@<UID:🚱🚃> ➡️  💠 [📱] Bob's iPhone Client (Address)
> Routing ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) via 🛣️  (SoloRoute)
> Received ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) @ 📱 Bob's iPhone (Device)@<UID:🚱🚃>
> Processing 📨 ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) @ 📱 Bob's iPhone (Device)@<UID:🚱🚃>
> Recipient Found ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply)@<UID:🚱🚃> == 📱 Bob's iPhone (Device)@<UID:🚱🚃>
> Creating ✉️  (ReprMessage) <UID:🚾🚫>
> Checking ✉️  (ReprMessage) 🔑 Matches 📱 Bob's iPhone (Device)@<UID:🚱🚃> root 🗝
> Matching 🔑 🛩  == 🜢  🗝
> ❌ Auth FAILED ✉️  (ReprMessage)


Oh oh! Why did Auth fail? We'll we can see from the debug that the 🔑 (VerifyKey) of the sender was matched to the 🗝 (Root VerifyKey) of the destination and they don't match. This client does not have sufficient permission to send a ReprMessage.

First lets take a look at the keys involved.

In [16]:
print(bob_device_client.keys)
print(bob_device.keys)

assert bob_device_client.verify_key != bob_device.root_verify_key

🔑 🛩
🔑 🜢🗝 🜢


Not to worry we have a solution, lets get a client which does have this permission.

In [17]:
bob_device_client = bob_device.get_root_client()

> Creating 📡 [📱] Bob's iPhone Client (DeviceClient)@<UID:🚱🚃>


Lets take a look again.

In [18]:
print(bob_device_client.keys)
print(bob_device.keys)

assert bob_device_client.verify_key == bob_device.root_verify_key

🔑 🛮
🔑 🜢🗝 🛮


Lets try sending the message again.

In [19]:
bob_device_client.send_immediate_msg_without_reply(
    msg=sy.ReprMessage(address=bob_device_client.address)
)

> Creating ✉️  (ReprMessage) <UID:🚆🚅>
> 📡 [📱] Bob's iPhone Client (DeviceClient)@<UID:🚱🚃> Signing ✉️  (ReprMessage) with 🛮
> Signing with 🛮
> Creating Signed ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) <UID:🚆🚅>
> Sending ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) 📡 [📱] Bob's iPhone Client (DeviceClient)@<UID:🚱🚃> ➡️  💠 [📱] Bob's iPhone Client (Address)
> Routing ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) via 🛣️  (SoloRoute)
> Received ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) @ 📱 Bob's iPhone (Device)@<UID:🚱🚃>
> Processing 📨 ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) @ 📱 Bob's iPhone (Device)@<UID:🚱🚃>
> Recipient Found ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply)@<UID:🚱🚃> == 📱 Bob's iPhone (Device)@<UID:🚱🚃>
> Creating ✉️  (ReprMessage) <UID:🚆🚅>
> Checking ✉️  (ReprMessage) 🔑 Matches 📱 Bob's iPhone (Device)@<UID:🚱🚃> root 🗝
> Matching 🔑 🛮  == 🛮  🗝
> ✅ Auth Succeeded ✉️  (ReprMessage) 🔑 == 🗝
Device:Bob's iPhone:<UID:ca85f85e-539d-4d48-a42a-9050f4613dc5>


Woohoo! 🎉

Okay so theres a lot going on but lets step through it.  
The ReprMessage is created by the Client, and then signed with the Client's SigningKey.
The SigningKey and VerifyKey are a pair and the VerifyKey is public and derived from the SigningKey.
When we call get_root_client() we update the Node with the newly generated key on the Client so that the Client will now have permission to execute root actions.

Behind every message type is a service which executes the message on a Node.
To run ReprMessage on our Device Node, we can see that during startup it adds a service to handle these kinds of messages:
```python
# common/node.py
self.immediate_services_without_reply.append(ReprService)
````

Not all actions / services require "root". To enable this a decorator is added like so:
```python
# repr_service.py
class ReprService(ImmediateNodeServiceWithoutReply):
    @staticmethod
    @service_auth(root_only=True)
```

Okay so Bob has root access to his own device, but he wants to share some data and compute resources of this device to someone else. So to do that he needs to create a "Sub Node" which will be a "VirtualMachine". Think of this as a partition or slice of his device which can be allocated memory, storage and compute.

In [20]:
bob_vm = sy.VirtualMachine(name="Bob's VM")

> Creating 🗃️ (MemoryStore) {}
> Creating 🍰 Bob's VM (VirtualMachine)@<UID:🚉🚀>


Since VirtualMachine is a Node (Server) it will need a Client to issue commands to it.
Lets make one.  
**note:** Why do we need a root client? The registration process is two way and the Registeree will need to update its address in response to a successful registration.

In [21]:
bob_vm_client = bob_vm.get_root_client()

> Creating 📡 [🍰] Bob's VM Client (VirtualMachineClient)@<UID:🚉🚀>


Okay so now Bob has two Nodes and their respective clients, but they know nothing of each other.
They both have addresses that only point to themselves.

In [22]:
print(bob_device_client.address.pprint)
print(bob_vm_client.address.pprint)

💠 [📱] Bob's iPhone Client (Address)
💠 [🍰] Bob's VM Client (Address)


Lets register Bob's vm with its device since the Device is higher up in the level of scope.

In [23]:
bob_device_client.register(client=bob_vm_client)

> Registering 📡 [🍰] Bob's VM Client (VirtualMachineClient)@<UID:🚉🚀> with 📡 [📱] Bob's iPhone Client (DeviceClient)@<UID:🚱🚃>
> Creating ✉️  (RegisterChildNodeMessage) <UID:🙼🚴>
> 📡 [📱] Bob's iPhone Client (DeviceClient)@<UID:🚱🚃> Signing ✉️  (RegisterChildNodeMessage) with 🛮
> Signing with 🛮
> ✉️  -> Proto 🔢
> Creating Signed ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) <UID:🙼🚴>
> Sending ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) 📡 [📱] Bob's iPhone Client (DeviceClient)@<UID:🚱🚃> ➡️  💠 [📱] Bob's iPhone Client (Address)
> Routing ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) via 🛣️  (SoloRoute)
> Received ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) @ 📱 Bob's iPhone (Device)@<UID:🚱🚃>
> Processing 📨 ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) @ 📱 Bob's iPhone (Device)@<UID:🚱🚃>
> Recipient Found ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply)@<UID:🚱🚃> == 📱 Bob's iPhone (Device)@<UID:🚱🚃>
> Creating ✉️  (RegisterChildNodeMessage) <UID:🙼🚴>
> ✉️  <- 🔢 Proto
> Checking ✉️  (RegisterChildNodeM

Whoa.. Okay lots happening. As you can see there are two messages and two Authentications. The first one is the `RegisterChildNodeMessage` which is dispatched to the address of the Device. Once it is received it stores the address of the registering Node and then dispatches a new `HeritageUpdateMessage` back to the sender of the first message.

**note:** This is not a reply message, this is a completely independent message that happens to be sent back to the sender's address.
```python
issubclass(RegisterChildNodeMessage, SignedImmediateSyftMessageWithoutReply)
issubclass(HeritageUpdateService, SignedImmediateSyftMessageWithoutReply)
```

You will also notice that the Messages turned into Protobufs and then Signed.  
Lets see how this works for `ReprMessage`.

1) ✉️  -> Proto 🔢

Every message that is sent requires and the following method signatures:
```python
def _object2proto(self) -> ReprMessage_PB:
def _proto2object(proto: ReprMessage_PB) -> "ReprMessage":
def get_protobuf_schema() -> GeneratedProtocolMessageType:
```

The get_protobuf_schema method will tell the caller what Protobuf class to use, and then the _object2proto method will be called to turn normal python into a protobuf message.

```python
# repr_service.py
def _object2proto(self) -> ReprMessage_PB:
    return ReprMessage_PB(
        msg_id=self.id.serialize(), address=self.address.serialize(),
    )
```

Any type which isnt a normal Protobuf primitive must be converted to a proto or serialized before being stored.
**note:** self.id and self.address also need to be serialized so this will call their `_object2proto` methods.  

At this point we are using code auto generated by `protoc` as per the build script:
```bash
$ ./scripts/build_proto.sh
```

Here is the .proto definition for `ReprMessage`

```c++
// repr_service.proto
syntax = "proto3";

package syft.core.node.common.service;

import "proto/core/common/common_object.proto";
import "proto/core/io/address.proto";

message ReprMessage {
    syft.core.common.UID msg_id = 1;
    syft.core.io.Address address = 2;
}
```


Once the message needs to be Deserialized the `_proto2object` method will be called.

```python
# repr_service.py
return ReprMessage(
    msg_id=_deserialize(blob=proto.msg_id),
    address=_deserialize(blob=proto.address),
)
```

Two things to pay attention to:
1) `RegisterChildNodeMessage` has caused Bob's Device Store has been updated with an entry representing Bob's VM Address  
2) `HeritageUpdateService` has caused Bob's VM to update its address to now include the `SpecificLocation` of Bob's Device.

**note:** The Address for Bob's VM Client inside the Store does not include the "Device" part of the "Address" (4 x Locations) since it isn't updated until after the HeritageUpdateService message is sent.

In [24]:
print(bob_device.store.pprint)
assert bob_vm_client.address.target_id.id in bob_device.store
print(bob_vm_client.address.pprint, bob_vm_client.address.target_id.id.emoji())

🗃️ (MemoryStore) {
  > <UID:344c64c5-46b2-4cc9-bfee-3b32e45079f1> <UID:🚉🚀> => 🗂️ (StorableObject) (💠 [🍰] Bob's VM Client (Address))
}
💠 [🍰📱] Bob's VM Client (Address) <UID:🚉🚀>


What about `SignedMessage`? If you read `message.py` you will see that all messages inherit from `SignedMessage` and as such contain the following fields.

```c++
message SignedMessage {
  syft.core.common.UID msg_id = 1;
  string obj_type = 2;
  bytes signature = 3;
  bytes verify_key = 4;
  bytes message = 5;
}
```

The actual message is serialized and stored in the `message` field and a hash of its bytes is calculated using the Client's `SigningKey`.

The contents of `message` are not encrypted and can be read at any time by simply deserializing them.

In [25]:
def get_signed_message_bytes() -> bytes:
    # return a signed message fixture containing the uid from get_uid
    message = {
        "msgId": {"value": "OSCFpuBSSeGS6TrP0S7Dkw=="},
        "objType": "syft.core.node.common.service.repr_service.ReprMessage",
        "signature": (
            "rltljJvXRhb3g4aRL6QJ1So1gbt9Vohp2BizxChbqhFkkeXPrmZmcE8RdSyRBTRnvOpAMXvjpe"
            + "nwEHK8olCg=="
        ),
        "verifyKey": "gf/M/DfEVS6KKh8iPTAQxO+IyDAB8H0zC9SXrS9Qjw8=",
        "message": (
            "eyJvYmpUeXBlIjogInN5ZnQuY29yZS5ub2RlLmNvbW1vbi5zZXJ2aWNlLnJlcHJfc2VydmljZS"
            + "5SZXByTWVzc2FnZSIsICJjb250ZW50IjogIntcIm1zZ0lkXCI6IHtcInZhbHVlXCI6IFwiT1"
            + "NDRnB1QlNTZUdTNlRyUDBTN0Rrdz09XCJ9LCBcImFkZHJlc3NcIjoge1wibmFtZVwiOiBcIl"
            + "Rlc3RcIiwgXCJoYXNWbVwiOiB0cnVlLCBcInZtXCI6IHtcImlkXCI6IHtcInZhbHVlXCI6IF"
            + "wiK3h1d1oxdTNURW0renVjQXF3b1ZGQT09XCJ9fX19In0="
        ),
    }
    envelope = {
        "objType": "syft.core.common.message.SignedImmediateSyftMessageWithoutReply",
        "content": json.dumps(message),
    }
    blob = bytes(json.dumps(envelope), "utf-8")
    return blob


In [26]:
sig_msg = sy.deserialize(blob=get_signed_message_bytes(), from_binary=True)

> Creating ✉️  (ReprMessage) <UID:🚄🚮>
> Creating Signed ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) <UID:🚄🚮>
> ✉️ 🔏 <- 🔢 Proto


We can get the nested message with a property called `message`

In [27]:
repr_msg = sig_msg.message
print(repr_msg.pprint, sig_msg.pprint)
print(repr_msg.address.pprint, sig_msg.address.pprint)
print(repr_msg.address.target_id.id.emoji(), sig_msg.address.target_id.id.emoji())
assert sig_msg.id == repr_msg.id
assert sig_msg.address == repr_msg.address

> Creating ✉️  (ReprMessage) <UID:🚄🚮>
✉️  (ReprMessage) ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply)
💠 [🍰] Test (Address) 💠 [🍰] Test (Address)
<UID:🙍🛖> <UID:🙍🛖>


Notice the UID's of `ReprMessage` and `SignedImmediateSyftMessageWithoutReply` are the same. So are the delivery address's.

But the original bytes are still available and serialization / deserialization or serde (ser/de) is bi-directional and reversible

In [28]:
assert repr_msg.serialize(to_binary=True) == sig_msg.serialized_message
print(repr_msg.pprint, " ⬅️ ", sig_msg.pprint)

✉️  (ReprMessage)  ⬅️  ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply)


In [29]:
from nacl.signing import SigningKey, VerifyKey
def get_signing_key() -> SigningKey:
    # return a the signing key used to sign the get_signed_message_bytes fixture
    key = "e89ff2e651b42393b6ecb5956419088781309d953d72bd73a0968525a3a6a951"
    return SigningKey(bytes.fromhex(key))

Lets try re-signing it with the same key it was signed with.

In [30]:
sig_msg_comp = repr_msg.sign(signing_key=get_signing_key())

> Signing with 🛳
> Creating Signed ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) <UID:🚄🚮>


In [31]:
signing_key = get_signing_key()
verify_key = signing_key.verify_key

print(f"SigningKey: {key_emoji(key=signing_key)}")
print(f"VerifyKey: {key_emoji(key=verify_key)}")
print(type(signing_key), type(verify_key))
print(f"🔑 {key_emoji(key=sig_msg.verify_key)} == {key_emoji(key=verify_key)} 🔑")
print(bytes(verify_key))

assert sig_msg_comp == sig_msg
assert sig_msg.verify_key == verify_key
assert VerifyKey(bytes(verify_key)) == verify_key

SigningKey: 🛭
VerifyKey: 🛳
<class 'nacl.signing.SigningKey'> <class 'nacl.signing.VerifyKey'>
🔑 🛳 == 🛳 🔑
b'\x81\xff\xcc\xfc7\xc4U.\x8a*\x1f"=0\x10\xc4\xef\x88\xc80\x01\xf0}3\x0b\xd4\x97\xad/P\x8f\x0f'


The message is signed with the `SigningKey`, a consistent `VerifyKey` is derived from the `SigningKey`. Both keys can be transformed to bytes and back easily.

Okay now Bob wants to protect his Device(s) and its / their VM(s). To do that he needs to add them to a higher level Node called a `Domain`.

In [32]:
bob_domain = sy.Domain(name="Bob's Domain")
bob_domain_client = bob_domain.get_root_client()

> Creating 🗃️ (MemoryStore) {}
> Creating 🏰 Bob's Domain (Domain)@<UID:🚅🚮>
> Creating 📡 [🏰] Bob's Domain Client (DomainClient)@<UID:🚅🚮>


Okay lets follow the same proceedure and link up these nodes.

In [33]:
print(bob_domain.address.pprint)

💠 [🏰] Bob's Domain (Address)


In [34]:
bob_domain_client.register(client=bob_device_client)

> Registering 📡 [📱] Bob's iPhone Client (DeviceClient)@<UID:🚱🚃> with 📡 [🏰] Bob's Domain Client (DomainClient)@<UID:🚅🚮>
> Creating ✉️  (RegisterChildNodeMessage) <UID:🚆🙿>
> 📡 [🏰] Bob's Domain Client (DomainClient)@<UID:🚅🚮> Signing ✉️  (RegisterChildNodeMessage) with 🚿
> Signing with 🚿
> ✉️  -> Proto 🔢
> Creating Signed ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) <UID:🚆🙿>
> Sending ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) 📡 [🏰] Bob's Domain Client (DomainClient)@<UID:🚅🚮> ➡️  💠 [🏰] Bob's Domain Client (Address)
> Routing ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) via 🛣️  (SoloRoute)
> Received ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) @ 🏰 Bob's Domain (Domain)@<UID:🚅🚮>
> Processing 📨 ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) @ 🏰 Bob's Domain (Domain)@<UID:🚅🚮>
> Recipient Found ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply)@<UID:🚅🚮> == 🏰 Bob's Domain (Domain)@<UID:🚅🚮>
> Creating ✉️  (RegisterChildNodeMessage) <UID:🚆🙿>
> ✉️  <- 🔢 Proto
> Checking ✉️  (RegisterChildNodeMessa

Thats interesting, we see theres two `HeritageUpdateMessage` that get sent. The address update is "Flowing" upward to the leaf VM nodes.

In [35]:
print(bob_vm.address.pprint)
print(bob_device.address.pprint)
print(bob_domain.address.pprint)
assert bob_domain_client.id == bob_device.address.domain.id
assert bob_device.id == bob_vm.address.device.id

💠 [🍰📱🏰] Bob's VM (Address)
💠 [📱🏰] Bob's iPhone (Address)
💠 [🏰] Bob's Domain (Address)


Now that the Nodes are aware of each other, we can send a message to any child node by dispatching a message on a parent Client and addressing the Child node.
**note:** We are changing the bob_vm root_verify_key because ReprMessage is a root message. We should change this example.

In [36]:
# TODO: Change this
bob_vm.root_verify_key = bob_domain_client.verify_key  # inject 📡🔑 as 📍🗝
bob_domain_client.send_immediate_msg_without_reply(
    msg=sy.ReprMessage(address=bob_vm.address)
)

> Creating ✉️  (ReprMessage) <UID:🚰🜑>
> 📡 [🏰] Bob's Domain Client (DomainClient)@<UID:🚅🚮> Signing ✉️  (ReprMessage) with 🚿
> Signing with 🚿
> Creating Signed ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) <UID:🚰🜑>
> Sending ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) 📡 [🏰] Bob's Domain Client (DomainClient)@<UID:🚅🚮> ➡️  💠 [🍰📱🏰] Bob's VM (Address)
> Routing ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) via 🛣️  (SoloRoute)
> Received ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) @ 🏰 Bob's Domain (Domain)@<UID:🚅🚮>
> Processing 📨 ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) @ 🏰 Bob's Domain (Domain)@<UID:🚅🚮>
> Recipient Not Found ↪️ ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply)@<UID:🚉🚀> != 🏰 Bob's Domain (Domain)@<UID:🚅🚮>
> Forwarding WithoutReply ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) to @<UID:🚉🚀>
<Address - Domain:<SpecificLocation:..e8b09>  Device:<SpecificLocation:..13dc5>, VM:<SpecificLocation:..079f1> in store doesnt have method send_immediate_msg_without_reply
'StorableObje

... To be continued by the next brave soul